In [5]:
PROJECT_FOLDER = '/content/drive/MyDrive/ColabNotebooks/!_ml_start_karpov/data/regularization/'

def connect_gdrive(PROJECT_FOLDER):
    from google.colab import drive
    import os

    drive.mount('/content/drive')
    os.chdir(PROJECT_FOLDER)
    print('Now current folder is: ', os.getcwd())

connect_gdrive(PROJECT_FOLDER)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Now current folder is:  /content/drive/MyDrive/ColabNotebooks/!_ml_start_karpov/data/regularization


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

from numpy.linalg import matrix_rank
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge


In [6]:
new_data = pd.read_csv('new_data.csv', index_col='id')
new_data.head(3)

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20
id,,,,,,,,,,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,1,1,1,1,1,...,0,0,0,0,0,1.500479,2.251437,3.378234,5.068969,7.605881
id2377394,0,930.399753,0,1.807119,6.498282,0,0,0,0,0,...,0,0,0,0,0,1.807119,3.265681,5.901475,10.664670,19.272331
id3858529,1,930.399753,0,6.392080,7.661527,1,1,1,1,1,...,0,0,0,0,0,6.392080,40.858690,261.172025,1669.432545,10671.146803


In [7]:
y_2 = new_data[['log_trip_duration']].copy()
X_2 = new_data.copy()
X_2.drop('log_trip_duration', axis=1, inplace=True)

X_2.head(3)

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20
id,,,,,,,,,,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,1,1,1,1,1,930.399753,...,0,0,0,0,0,1.500479,2.251437,3.378234,5.068969,7.605881
id2377394,0,930.399753,0,1.807119,0,0,0,0,0,930.399753,...,0,0,0,0,0,1.807119,3.265681,5.901475,10.664670,19.272331
id3858529,1,930.399753,0,6.392080,1,1,1,1,1,930.399753,...,0,0,0,0,0,6.392080,40.858690,261.172025,1669.432545,10671.146803


1. Как выглядит трансформация тестовых данных на этапе предсказания модели, которая была построена с применением техник масштабирования и регуляризации?
2. При использовании кросс-валидации, то MinMax или StandardScaler рекомендуется делать на всей выборке обучающих данных, либо отдельно внутри каждого фолда?
3. Какие рекомендации по выбору той или иной техники масштабирования признаков? MinMax или StandardScaler или остальные из арсенала sklearn?

In [ ]:
scaler_x = MinMaxScaler()
scaler_x.fit(X_2)
X_2_transformed = scaler_x.transform(X_2)
X_2_transformed[:3]

In [9]:
scaler_y = MinMaxScaler()
scaler_y.fit(y_2)
y_2_transformed = scaler_y.transform(y_2)

In [30]:
alpha = 0.05

cv_scores_ridge= cross_validate(
    Ridge(alpha),
    X_2_transformed, 
    y_2_transformed,
    cv=4,
    return_train_score=True,
    scoring='neg_mean_squared_error'
)

cv_scores_ridge = np.mean(cv_scores_ridge['test_score'] * -1)

print(f"MSLE на Кросс-валидации: {round(cv_scores_ridge, 3)}")

MSLE на Кросс-валидации: 0.002


In [4]:
for alpha in np.linspace(1e-5, 1e-4, 10):
    print(alpha)

1e-05
2e-05
3.0000000000000004e-05
4e-05
5e-05
6e-05
7.000000000000001e-05
8e-05
9e-05
0.0001


In [ ]:
for alpha in np.linspace(0.0, 0.5, 51):
    cv_scores_ridge= cross_validate(
        Ridge(alpha),
        X_2_transformed, 
        y_2_transformed,
        cv=4,
        return_train_score=True,
        scoring='neg_mean_squared_error'
    )

    cv_scores_ridge = np.mean(cv_scores_ridge['test_score'] * -1)

    print(f"MSLE на Кросс-валидации: {round(cv_scores_ridge, 3)}, alpha = {alpha:.3f}")

In [ ]:
for alpha in np.linspace(1e-5, 1e-4, 100):
    cv_scores_lasso = cross_validate(
        Lasso(alpha),
        X_2_transformed, 
        y_2_transformed,
        cv=4,
        return_train_score=True,
        scoring='neg_mean_squared_error'
    )

    cv_scores_lasso = np.mean(cv_scores_lasso['test_score'] * -1)

    print(f"MSLE на Кросс-валидации: {round(cv_scores_lasso, 3)}, alpha = {alpha:.5f}")